In [1]:
import pandas as pd
import openpyxl

In [47]:
# Read xlsx file and conver to dataframe
df = pd.read_excel('Anulaciones.xlsx', sheet_name='Report')

In [43]:
def filter_dataframe_vida(df, year, riesgo, vida=False):
    # Initialize an empty list to store the selected DataFrames
    selected_dfs = []

    # Flags to track whether "TOTAL:" and "Contrato:" are found
    total_found = False
    contrato_found = False

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Check conditions for each row
        if (
            df.iloc[i, 2] == "TOTAL:" and
            df.iloc[i, 3] == year and
            (df.iloc[i, 7]).split()[0] == "VIDA" and
            not (df.iloc[i, 7]).split()[0] == "CAUCION" and
            df.iloc[i, 4] == riesgo
        ):
            total_found = True
            contrato_found = False
        elif df.iloc[i, 2] == "Contrato:":
            contrato_found = True
            total_found = False
    
        # Select rows between "TOTAL:" and "Contrato:"
        if total_found and not contrato_found:
            selected_dfs.append(df.iloc[i])

    # Check if there are selected DataFrames
    if selected_dfs:
        # Concatenate the list of selected DataFrames into a single DataFrame
        result_df = pd.concat(selected_dfs, axis=1).T

        # Filter out rows that start with specific words in the first column
        result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]

        # Filter out rows that in the third column start with specific words
        result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones', 'Anulaciones'))]

        # Remove columns with all NaN values
        result_df = result_df.dropna(axis=1, how='all')

        # Change the column names for letters. Example: "Unamed: 0" to "A"
        result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

        return result_df
    else:
        print("No matching rows found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_filtered_df = filter_dataframe_vida(df, 2023, 'EXCEDENTE')
if your_filtered_df is not None:
    print(your_filtered_df)
"""


def filter_dataframe_patrimoniales(df, year, riesgo, vida=False):
    # Initialize an empty list to store the selected DataFrames
    selected_dfs = []

    # Flags to track whether "TOTAL:" and "Contrato:" are found
    total_found = False
    contrato_found = False

    # Iterate through the DataFrame
    for i in range(len(df)):
        # Check conditions for each row
        if (
            df.iloc[i, 2] == "TOTAL:" and
            df.iloc[i, 3] == year and
            not (df.iloc[i, 7]).split()[0] == "VIDA" and
            not (df.iloc[i, 7]).split()[0] == "CAUCION" and
            df.iloc[i, 4] == riesgo
        ):
            total_found = True
            contrato_found = False
        elif df.iloc[i, 2] == "Contrato:":
            contrato_found = True
            total_found = False
    
        # Select rows between "TOTAL:" and "Contrato:"
        if total_found and not contrato_found:
            selected_dfs.append(df.iloc[i])

    # Check if there are selected DataFrames
    if selected_dfs:
        # Concatenate the list of selected DataFrames into a single DataFrame
        result_df = pd.concat(selected_dfs, axis=1).T

        # Filter out rows that start with specific words in the first column
        result_df = result_df[~result_df.iloc[:, 0].astype(str).str.startswith(('Póliza', 'Operador:'))]

        # Filter out rows that in the third column start with specific words
        result_df = result_df[~result_df.iloc[:, 2].astype(str).str.startswith(('REGIONAL', 'Listado', 'Emisiones', 'Anulaciones'))]

        # Remove columns with all NaN values
        result_df = result_df.dropna(axis=1, how='all')

        # Change the column names for letters. Example: "Unamed: 0" to "A"
        result_df.columns = [chr(65 + i) for i in range(len(result_df.columns))]

        return result_df
    else:
        print("No matching rows found.")
        return None
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_filtered_df = filter_dataframe_patrimoniales(df, 2023, 'EXCEDENTE')
if your_filtered_df is not None:
    print(your_filtered_df)
"""


def remove_totals(df):
    # Remove all the rows that start with "TOTAL:" in the second column
    result_df = df[~df.iloc[:, 1].astype(str).str.startswith('TOTAL:')]
    
    # Remove all empty rows
    result_df = result_df.dropna(how='all')

    return result_df
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
your_processed_df = process_dataframe(your_dataframe)
print(your_processed_df)
"""


def calculate_sums(df):
    # Suma los valores numéricos de las columnas H, la columna L y la columna M,
    # devuélveme los valores y almacénalos en variables "prima", "importe_comision" y "a_favor" respectivamente
    prima = pd.to_numeric(df['H'], errors='coerce').fillna(0).sum()
    importe_comision = pd.to_numeric(df['L'], errors='coerce').fillna(0).sum()
    a_favor = pd.to_numeric(df['M'], errors='coerce').fillna(0).sum()

    return prima, importe_comision, a_favor
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
your_processed_df = process_dataframe(your_dataframe)
prima_value, importe_comision_value, a_favor_value = calculate_sums(your_processed_df)
print(prima_value, importe_comision_value, a_favor_value)
"""


"\n# Example usage\n# Replace 'your_dataframe' with the actual DataFrame variable name\nyour_processed_df = process_dataframe(your_dataframe)\nprima_value, importe_comision_value, a_favor_value = calculate_sums(your_processed_df)\nprint(prima_value, importe_comision_value, a_favor_value)\n"

In [44]:
def process_and_sum_vida(df, year, riesgo):
    # Step 1: Filter the DataFrame
    filtered_df = filter_dataframe_vida(df, year, riesgo)
    
    # Step 2: Remove totals from the filtered DataFrame
    processed_df = remove_totals(filtered_df)
    
    # Step 3: Calculate the sums
    sums_result = calculate_sums(processed_df)
    
    return sums_result
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_sums_result = process_and_sum_vida(df, 2023, 'EXCEDENTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])
"""
def process_and_sum_patrimoniales(df, year, riesgo):
    # Step 1: Filter the DataFrame
    filtered_df = filter_dataframe_patrimoniales(df, year, riesgo)
    
    # Step 2: Remove totals from the filtered DataFrame
    processed_df = remove_totals(filtered_df)
    
    # Step 3: Calculate the sums
    sums_result = calculate_sums(processed_df)
    
    return sums_result
"""
# Example usage
# Replace 'your_dataframe' with the actual DataFrame variable name
# Replace 2022 and 'EXCEDENTE' with the desired values for year and riesgo
your_sums_result = process_and_sum_patrimoniales(df, 2023, 'EXCEDENTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])
"""


'\n# Example usage\n# Replace \'your_dataframe\' with the actual DataFrame variable name\n# Replace 2022 and \'EXCEDENTE\' with the desired values for year and riesgo\nyour_sums_result = process_and_sum_patrimoniales(df, 2023, \'EXCEDENTE\')\nprint("Prima:", your_sums_result[0])\nprint("Importe Comision:", your_sums_result[1])\nprint("A Favor:", your_sums_result[2])\n'

In [45]:
your_sums_result = process_and_sum_patrimoniales(df, 2021, 'CUOTA PARTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])

Prima: 222174885.0
Importe Comision: 67096913.0
A Favor: 155077972.0


In [48]:
your_sums_result = process_and_sum_patrimoniales(df, 2020, 'EXCEDENTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])

Prima: 193979440.0
Importe Comision: 60909352.0
A Favor: 133070088.0


In [ ]:
your_sums_result = process_and_sum_patrimoniales(df, 2020, 'CUOTA PARTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])

In [ ]:
your_sums_result = process_and_sum_patrimoniales(df, 2022, 'EXCEDENTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])

In [ ]:
your_sums_result = process_and_sum_patrimoniales(df, 2022, 'CUOTA PARTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])

In [ ]:
your_sums_result = process_and_sum_patrimoniales(df, 2023, 'EXCEDENTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])

In [ ]:
your_sums_result = process_and_sum_patrimoniales(df, 2023, 'CUOTA PARTE')
print("Prima:", your_sums_result[0])
print("Importe Comision:", your_sums_result[1])
print("A Favor:", your_sums_result[2])

In [39]:
result_df = filter_dataframe_patrimoniales(df, 2023, 'CUOTA PARTE')
result_df.to_excel('anulaciones QS.xlsx', index=False)